In [1]:
gROOT->ProcessLine(".L common/gallery_includes.h+O");
gROOT->ProcessLine(".L common/larsoftobj_includes.h+O");

Info in <TUnixSystem::ACLiC>: creating shared library /uboone/app/users/wketchum/notebooks/Sample_Notebooks/./common/gallery_includes_h.so
Info in <TUnixSystem::ACLiC>: creating shared library /uboone/app/users/wketchum/notebooks/Sample_Notebooks/./common/larsoftobj_includes_h.so


In [2]:
//setup our input file and input tag
std::vector<std::string> filenames;
TFile *f_output;

TNtuple *nt;
TCanvas canvas("canvas","MyCanvas");

art::InputTag vtx_tag;
art::InputTag pfp_tag;
art::InputTag gen_tag;

//geometry
float X_LENGTH=256.0;
float Y_HALF_HEIGHT=116.5;
float Z_LENGTH=1037.0;

In [3]:
%%cpp -d
void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const&,
                    int ntab=0);

void PrintDaughters(size_t i_pfp,
                    std::vector<recob::PFParticle> const& pfp_vec,
                    int ntab)
{
    //std::cout << "\tN_daughters=" << pfp_vec[i_pfp].NumDaughters() << std::endl;
    for(size_t i_d=0; i_d<pfp_vec[i_pfp].NumDaughters(); ++i_d){
        for(int it=0; it<ntab; ++it) std::cout << "\t";
        std::cout << "Found daughter particle id=" << pfp_vec[i_pfp].Daughters()[i_d] 
                  << ", code=" << pfp_vec[pfp_vec[i_pfp].Daughters()[i_d]].PdgCode() << std::endl;
        PrintDaughters(pfp_vec[i_pfp].Daughters()[i_d],pfp_vec,ntab+1);
    }
    return;
}

In [4]:
filenames = 
{
    "/pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_996_20180207T201546_gen2_d63adf8e-d13a-455f-aef5-fb1518a37df7_20180212T065449_reco1_20180212T081208_reco2.root"
};

f_output = new TFile("/uboone/data/users/wketchum/tmp.root","RECREATE");

nt = new TNtuple("nt","MyNtuple","run:event:nu_x:nu_y:nu_z:vtx_x:vtx_y:vtx_z:vtx_dist:pfp_primary:pfp_pdg");

vtx_tag = { "pandoraNu" };
pfp_tag = { "pandoraNu" };
gen_tag = { "generator" };


In [5]:
for(gallery::Event ev(filenames) ; !ev.atEnd(); ev.next()) {
/*
    std::cout << "Processing " 
              << "Run " << ev.eventAuxiliary().run() << ", "
              << "Event " << ev.eventAuxiliary().event() << endl;
*/  
    
    auto const& mctruth_handle = ev.getValidHandle<std::vector<simb::MCTruth>>(gen_tag);
    auto const& truth_vec(*mctruth_handle);

    if(truth_vec.size()==0){
        std::cout << "ERROR! truth_vec size is " << truth_vec.size() << std::endl;
        break;
    }
       
    auto const& nu = truth_vec[0].GetNeutrino();
    if(nu.Nu().Vx()<0.0+10.0 || nu.Nu().Vx()>X_LENGTH-10.0 ||
       nu.Nu().Vy()<-1*Y_HALF_HEIGHT+20.0 || nu.Nu().Vy()>Y_HALF_HEIGHT-20.0 ||
       nu.Nu().Vz()<0.0 || nu.Nu().Vz()>Z_LENGTH-10.0
       )
        continue;

    std::cout << "Neutrino position is (" 
              << nu.Nu().Vx() << ","
              << nu.Nu().Vy() << ","
              << nu.Nu().Vz() << ")"
              << std::endl;

    geo::Point_t nu_position(nu.Nu().Vx(),nu.Nu().Vy(),nu.Nu().Vz());
        
    auto const& pfp_handle = ev.getValidHandle<std::vector<recob::PFParticle>>(pfp_tag);
    auto const& pfp_vec(*pfp_handle);
    std::cout << "There are " << pfp_vec.size() << " PFParticles in this event." << std::endl;

    for (auto const& pfp : pfp_vec){
        if(!pfp.IsPrimary()) continue;
        std::cout << "Found primary particle id=" << pfp.Self() << ", code=" << pfp.PdgCode() << std::endl;
        PrintDaughters(pfp.Self(),pfp_vec,1);
    }
    
    
    auto const& vtx_handle = ev.getValidHandle<std::vector<recob::Vertex>>(vtx_tag);
    auto const& vtx_vec(*vtx_handle);
    std::cout << "There are " << vtx_vec.size() << " vertices in this event." << std::endl;
    /*  
    for(auto const& vtx : vtx_vec){
        auto const& pos = vtx.position();
        std::cout << "\tPosition (x,y,z)="
                  << "(" << pos.X() << ","
                  << "" << pos.Y() << ","
                  << "" << pos.Z() << ")"
                  << std::endl;
    }
    */
    
    //auto const& pfp_vtx_handle = ev.getValidHandle<art::Assns<recob::PFParticle,recob::Vertex,void> >(pfp_tag);
    //auto const& pfp_vtx_assn_vec(*pfp_vtx_handle);
    //std::cout << "Assns: " << pfp_vtx_assn_vec.size();
    
    art::FindOne<recob::Vertex> vtx_per_pfp(pfp_handle,ev,pfp_tag);
    double min_dist=9e9; int i_pfp_nu=-1; geo::Point_t pfp_position;
    
    for(size_t i_pfp=0; i_pfp<pfp_vec.size(); ++i_pfp)
    {
        
        auto vtxref = vtx_per_pfp.at(i_pfp);
        if(vtxref.isValid()){
            auto const& vtx(vtxref.ref());
            std::cout << "For pfparticle " << i_pfp << " found vertex at "
                  << "(" << vtx.position().X() << ","
                  << ""  << vtx.position().Y() << ","
                  << ""  << vtx.position().Z() << ")"
                  << std::endl;
            
            if(sqrt((vtx.position()-nu_position).Mag2()) < min_dist){
                min_dist = sqrt((vtx.position()-nu_position).Mag2());
                i_pfp_nu = i_pfp;
                pfp_position = vtx.position();
            }
        }
        
            
    }
    
    nt->Fill(ev.eventAuxiliary().run(),ev.eventAuxiliary().event(),
             nu_position.X(),nu_position.Y(),nu_position.Z(),
             pfp_position.X(),pfp_position.Y(),pfp_position.Z(),
             min_dist,
             pfp_vec[i_pfp_nu].IsPrimary(),pfp_vec[i_pfp_nu].PdgCode());
    
    
}//end loop over events


Warning in <TClass::Init>: no dictionary for class art::DoNotRecordParents is available
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoConstants.h
   requested to autoload type larlite::geo::View_t
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoConstants.h
   requested to autoload type larlite::geo::SigType_t
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::TPCID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::PlaneID
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for GeoTypes.h
   requested to autoload type larlite::geo::WireID


Successfully opened file /pnfs/uboone/mc/uboone/reconstructed/prod_v06_26_07/prodgenie_bnb_intrinsic_nue_cosmic_uboone_MCC8Tune3_reco/reco/prodgenie_bnb_intrinsic_nue_cosmic_uboone_996_20180207T201546_gen2_d63adf8e-d13a-455f-aef5-fb1518a37df7_20180212T065449_reco1_20180212T081208_reco2.root


Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for larlite_base.h
   requested to autoload type larlite::Message
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for larlite_base.h
   requested to autoload type larlite::larlite_base
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for DetectorProperties.h
   requested to autoload type larutil::LArUtilException
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for DetectorProperties.h
   requested to autoload type larutil::LArUtilConfig
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for DetectorProperties.h
   requested to autoload type larutil::DetectorProperties
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ElecClock.h
   requested to autoload type larutil::ElecClock
Error in cling::AutoloadingVisitor::InsertInto

Neutrino position is (55.8618,36.0213,373.634)
There are 14 PFParticles in this event.
Found primary particle id=8, code=14
	Found daughter particle id=0, code=13
Found primary particle id=9, code=14
	Found daughter particle id=1, code=13
Found primary particle id=10, code=14
	Found daughter particle id=3, code=13
Found primary particle id=11, code=12
	Found daughter particle id=2, code=11
Found primary particle id=12, code=12
	Found daughter particle id=4, code=11
	Found daughter particle id=6, code=13
	Found daughter particle id=7, code=11
Found primary particle id=13, code=14
	Found daughter particle id=5, code=13
There are 14 vertices in this event.
For pfparticle 0 found vertex at (238.428,-90.7326,402.857)
For pfparticle 1 found vertex at (127.015,20.0648,466.033)
For pfparticle 2 found vertex at (51.8369,27.99,403)
For pfparticle 3 found vertex at (235.465,68.9942,343.147)
For pfparticle 4 found vertex at (27.3801,47.4868,387.85)
For pfparticle 5 found vertex at (161.941,87.55,7

In [6]:
%jsroot on
canvas.cd();
nt->Draw("pfp_primary","vtx_dist<10");
canvas.Draw();